# Row & Column class

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/20 09:17:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 09:17:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Row Class

- 하나의 row 데이터를 만들 때 사용할 수 있다

In [2]:
from pyspark.sql import Row

### Create a Row Object

In [3]:
row1=Row("James",40)

row2=Row(name="Alice", age=11)

Person = Row("name", "age")
p1=Person("James", 40)
p2=Person("Alice", 35)

In [4]:
print(row1[0], row1[1])
print(row2.name, row2.age)
print(p1.name, p1.age)

James 40
Alice 11
James 40


### Using Row class on PySpark RDD

In [5]:
from pyspark.sql import Row

data = [Row(name="James,,Smith",lang=["Java","Scala","C++"],state="CA"), 
    Row(name="Michael,Rose,",lang=["Spark","Java","C++"],state="NJ"),
    Row(name="Robert,,Williams",lang=["CSharp","VB"],state="NV")]

rdd=spark.sparkContext.parallelize(data)
print(rdd.collect())

[Row(name='James,,Smith', lang=['Java', 'Scala', 'C++'], state='CA'), Row(name='Michael,Rose,', lang=['Spark', 'Java', 'C++'], state='NJ'), Row(name='Robert,,Williams', lang=['CSharp', 'VB'], state='NV')]


### Using Row class on PySpark DataFrame

In [6]:
df=spark.createDataFrame(data)
df.show()

+----------------+------------------+-----+
|            name|              lang|state|
+----------------+------------------+-----+
|    James,,Smith|[Java, Scala, C++]|   CA|
|   Michael,Rose,|[Spark, Java, C++]|   NJ|
|Robert,,Williams|      [CSharp, VB]|   NV|
+----------------+------------------+-----+



## Column Class

- 데이터 처리/분석을 하다보면 컬럼 단위로 데이터를 다루는 경우가 많다
- Spark DataFrame은 특정 컬럼에 접근하면 Column 클래스 객체를 돌려주고, Column 클래스 객체만의 데이터 처리/분석에 유용한 속성과 메서드를 제공해준다
- Column Class 에 정의된 메서드 말고도 pyspark.sql.functions에서 제공하는 함수도 있다

### Create Column Class Object

In [7]:
from pyspark.sql.functions import lit

colObj = lit("myspark")
type(colObj)

pyspark.sql.column.Column

###  Access the Column from DataFrame

In [8]:
data=[("James",23),("Ann",40)]
columns = ["name.fname", "gender"]

df=spark.createDataFrame(data, schema=columns)

df.printSchema()
df.show()

root
 |-- name.fname: string (nullable = true)
 |-- gender: long (nullable = true)

+----------+------+
|name.fname|gender|
+----------+------+
|     James|    23|
|       Ann|    40|
+----------+------+



In [9]:
print(type(df.gender))
print(type(df["gender"]))
print(type(df["`name.fname`"])) # backticks for accessing column with dot

from pyspark.sql.functions import col
print(type(col("gender")))

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>


In [10]:
df.select(col("gender")).show()

+------+
|gender|
+------+
|    23|
|    40|
+------+



### PySpark Column Operators

In [11]:
data=[(100,2,1),(200,3,4),(300,4,4)]
columns = ["col1", "col2", "col3"]
df=spark.createDataFrame(data, schema=columns)
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
| 100|   2|   1|
| 200|   3|   4|
| 300|   4|   4|
+----+----+----+



In [12]:
df.select(df.col1 + df.col2).show()
df.select(df.col1 - df.col2).show() 
df.select(df.col1 * df.col2).show()
df.select(df.col1 / df.col2).show()
df.select(df.col1 % df.col2).show()

df.select(df.col2 > df.col3).show()
df.select(df.col2 < df.col3).show()
df.select(df.col2 == df.col3).show()

+-------------+
|(col1 + col2)|
+-------------+
|          102|
|          203|
|          304|
+-------------+

+-------------+
|(col1 - col2)|
+-------------+
|           98|
|          197|
|          296|
+-------------+

+-------------+
|(col1 * col2)|
+-------------+
|          200|
|          600|
|         1200|
+-------------+

+-----------------+
|    (col1 / col2)|
+-----------------+
|             50.0|
|66.66666666666667|
|             75.0|
+-----------------+

+-------------+
|(col1 % col2)|
+-------------+
|            0|
|            2|
|            0|
+-------------+

+-------------+
|(col2 > col3)|
+-------------+
|         true|
|        false|
|        false|
+-------------+

+-------------+
|(col2 < col3)|
+-------------+
|        false|
|         true|
|        false|
+-------------+

+-------------+
|(col2 = col3)|
+-------------+
|        false|
|        false|
|         true|
+-------------+



### PySpark Column Functions

In [13]:
data=[("James","Bond","100",None),
      ("Ann","Varsa","200",'F'),
      ("Tom Cruise","XXX","400",''),
      ("Tom Brand",None,"400",'M')]
columns=["fname","lname","id","gender"]

df=spark.createDataFrame(data,columns)
df.show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



#### alias() & name() – Set’s name to Column

In [14]:
from pyspark.sql.functions import concat_ws

# (concat_ws() 는 두 문자열 컬럼을 붙이기 위한 함수이다)

# Column.alias(<원하는 컬럼명>)
# Column.name(<원하는 컬럼명>)

df.select(concat_ws(" ", df.fname, df.lname).alias("full_name"), df.id, df.gender).show()

+--------------+---+------+
|     full_name| id|gender|
+--------------+---+------+
|    James Bond|100|  null|
|     Ann Varsa|200|     F|
|Tom Cruise XXX|400|      |
|     Tom Brand|400|     M|
+--------------+---+------+



#### asc() & desc() – Sort the DataFrame columns by Ascending or Descending order

In [15]:
# Column.asc(): 컬럼 기준 오름차순 정렬

df.sort(df.fname.asc()).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|     James| Bond|100|  null|
| Tom Brand| null|400|     M|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



#### cast() & astype() – Used to convert the data Type

In [16]:
df.select(df.fname,df.id.cast("int")).printSchema()

root
 |-- fname: string (nullable = true)
 |-- id: integer (nullable = true)



#### between() – Returns a Boolean expression when a column values in between lower and upper bound

In [17]:
# Column.between(lower, upper)
df.select(df.id.between(0, 300)).show()

+---------------------------+
|((id >= 0) AND (id <= 300))|
+---------------------------+
|                       true|
|                       true|
|                      false|
|                      false|
+---------------------------+



In [18]:
df.filter(df.id.between(0, 300)).show()

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
|  Ann|Varsa|200|     F|
+-----+-----+---+------+



#### contains() – Checks if a DataFrame column value contains a a value specified in this function

In [19]:
# Column.contains()
df.filter(df.fname.contains("Brand")).show()

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+



#### startswith() & endswith() – Checks if the value of the DataFrame Column starts and ends with a String respectively

In [20]:
# Column.startswith()
# Column.endswith()

df.filter(df.fname.startswith("T")).show()
df.filter(df.fname.endswith("Brand")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+



#### isNull() & isNotNull() – Checks if the DataFrame column has NULL or non NULL values

In [21]:
# Column.isNull()
# Column.isNotNull()

df.filter(df.lname.isNull()).show()
df.filter(df.lname.isNotNull()).show()

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



#### like() & rlike() – Similar to SQL LIKE expression

In [22]:
# Column.like()
# Column.rlike()

df.filter(df.fname.like("%ise")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



#### substr() – Returns a Column after getting sub string from the Column

In [23]:
# Column.substr()


df.select(df.fname.substr(1,2).alias("substr")).show()

+------+
|substr|
+------+
|    Ja|
|    An|
|    To|
|    To|
+------+



#### when() & otherwise() – It is similar to SQL Case

In [24]:
# when() can only be applied on a Column previously generated by when() function
from pyspark.sql.functions import when

df.select(df.gender, when(df.gender=="M","Male").when(df.gender=="F", "Female") \
                                                  .when(df.gender==None, "None") \
                                                  .otherwise(df.gender).alias("full_gender")).show()

+------+-----------+
|gender|full_gender|
+------+-----------+
|  null|       null|
|     F|     Female|
|      |           |
|     M|       Male|
+------+-----------+



#### isin() – Check if value presents in a List

In [25]:
li=["100","200"]
df.filter(df.id.isin(li)) \
  .show()

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
|  Ann|Varsa|200|     F|
+-----+-----+---+------+

